<a href="https://colab.research.google.com/github/abrahamjenie/Programming-for-Data-Analysis-CA1-Web-Scraping/blob/main/SalaryData_WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#Importing Packages
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time

In [13]:
url="https://www.salary.com/research/ie-salary/alternate/data-scientist-analytics-intermediate-salary/ie/dublin"
r = requests.get(url)
print(r) #status code is 200 

<Response [200]>


In [14]:
r.content
1

1

In [15]:
soup=BeautifulSoup(r.content,'html.parser')
script=soup.find_all('script')[5].text
script

'\r\n    {\r\n    "@context": "http://schema.org",\r\n    "@type": "Occupation",\r\n    "name": "Data Scientist/Analytics , Intermediate",\r\n    "mainEntityOfPage": {\r\n    "@type": "WebPage",\r\n    "lastReviewed": "2021-03-19T00:00:00Z"\r\n    },\r\n    "description": "A Data Scientist is a professional who extensively works with Big Data in order to derive valuable business insights from it.\r\nHowever, data scientists and predictive analytics professionals (PAPs) are more likely to hold an advanced degree.\r\nDay-to-day activities of a Data Scientist sometimes can be predictable, and sometimes they are something out of the ordinary.\r\n",\r\n    "estimatedSalary": [{\r\n    "@type": "MonetaryAmountDistribution",\r\n    "name": "base",\r\n    "currency": "EUR",\r\n    "duration": "P1Y",\r\n    "percentile25": "31559",\r\n    "median": "39009",\r\n    "percentile75": "46472"\r\n    }]\r\n        \r\n            ,\r\n            "occupationLocation": [{\r\n            "@type": "City

In [16]:
json_data=json.loads(script,strict=False) #strict=False is used because there are control characters like '\r','\n'
json_data

{'@context': 'http://schema.org',
 '@type': 'Occupation',
 'name': 'Data Scientist/Analytics , Intermediate',
 'mainEntityOfPage': {'@type': 'WebPage',
  'lastReviewed': '2021-03-19T00:00:00Z'},
 'description': 'A Data Scientist is a professional who extensively works with Big Data in order to derive valuable business insights from it.\r\nHowever, data scientists and predictive analytics professionals (PAPs) are more likely to hold an advanced degree.\r\nDay-to-day activities of a Data Scientist sometimes can be predictable, and sometimes they are something out of the ordinary.\r\n',
 'estimatedSalary': [{'@type': 'MonetaryAmountDistribution',
   'name': 'base',
   'currency': 'EUR',
   'duration': 'P1Y',
   'percentile25': '31559',
   'median': '39009',
   'percentile75': '46472'}],
 'occupationLocation': [{'@type': 'City', 'name': 'Dublin, Ireland'}]}

In [17]:
meta_url=soup.find('meta',property="og:url")
meta_url

<meta content="https://www.salary.com/research/ie-salary/alternate/data-scientist-analytics-intermediate-salary/ie/dublin" property="og:url"/>

In [18]:
job_designation=json_data['name']
job_desc=json_data['description']
percentile25_salary=json_data['estimatedSalary'][0]['percentile25']
median_salary=json_data['estimatedSalary'][0]['median']
percentile75_salary=json_data['estimatedSalary'][0]['percentile75']
salary_type=json_data['estimatedSalary'][0]['name']
currency=json_data['estimatedSalary'][0]['currency']
salary_duration=json_data['estimatedSalary'][0]['duration']
location=json_data['occupationLocation'][0]['name']
last_reviewed_datetime=json_data['mainEntityOfPage']['lastReviewed']
weblink=meta_url['content']

print(job_designation,percentile25_salary,median_salary,percentile75_salary,salary_type,currency,salary_duration,location,last_reviewed_date,meta_url,weblink)

Data Scientist/Analytics , Intermediate 31559 39009 46472 base EUR P1Y Dublin, Ireland 2021-03-19T00:00:00Z <meta content="https://www.salary.com/research/ie-salary/alternate/data-scientist-analytics-intermediate-salary/ie/dublin" property="og:url"/> https://www.salary.com/research/ie-salary/alternate/data-scientist-analytics-intermediate-salary/ie/dublin


In [19]:
cities=['dublin','cork','galway','limerick','shannon-west','waterford']
job_roles=['data-architect-ii','personal-assistant-to-ceo','operational-risk-analyst-ii','entry-hr-generalist','senior-risk-analyst','asset-and-liability-risk-analyst','junior-accountant','financial-accountant-i','it-user-support-manager','data-scientist-analytics-senior-specialist','life-sciences-research-scientist-iii','hr-assistant-ii','data-scientist-analytics-intermediate','entry-financial-analyst','entry-accountant','graphic-designer-ii','instructional-designer','healthcare-services-planning-director','warehouse-operations-supervisor','database-warehouse-programmer','warehouse-helper']
data={}
salary_df=pd.DataFrame(data)
for role in job_roles:
  for city in cities:
    alternate = 'https://www.salary.com/research/ie-salary/alternate/{}-salary/ie/{}'
    benchmark = 'https://www.salary.com/research/ie-salary/benchmark/{}-salary/ie/{}'

    url_alt = alternate.format(role, city)
    url_bench= benchmark.format(role, city)
    r_alt = requests.get(url_alt)
    r_bench=requests.get(url_bench)

    if r_alt.status_code==200 and r_bench.status_code!=200:
      r=requests.get(url_alt)
    elif r_bench.status_code==200 and r_alt.status_code!=200:
      r=requests.get(url_bench)
    else:
      print("HTTP request failed for {} and {}".format(url_alt,url_bench))
      continue

        
    soup=BeautifulSoup(r.content,'html.parser')
    script=soup.find_all('script')[5].text
    
    json_data=json.loads(script,strict=False)
    
    meta_url=soup.find('meta',property="og:url")
    
    new_row={'job_designation':json_data['name'],'percentile25_salary':json_data['estimatedSalary'][0]['percentile25'],
             'median_salary':json_data['estimatedSalary'][0]['median'],'percentile75_salary':json_data['estimatedSalary'][0]['percentile75'],
             'salary_type':json_data['estimatedSalary'][0]['name'],'currency':json_data['estimatedSalary'][0]['currency'],
             'salary_duration':json_data['estimatedSalary'][0]['duration'],'location':json_data['occupationLocation'][0]['name'],
             'last_reviewed_datetime':json_data['mainEntityOfPage']['lastReviewed'],'weblink':meta_url['content'],'job_desc':json_data['description']}
    salary_df = salary_df.append(new_row, ignore_index=True)
print(salary_df)

       job_designation percentile25_salary median_salary percentile75_salary  \
0    Data Architect II               42167         50115               58800   
1    Data Architect II               39972         47506               55739   
2    Data Architect II               39035         46393               54433   
3    Data Architect II               39420         46850               54969   
4    Data Architect II               38646         45930               53890   
..                 ...                 ...           ...                 ...   
121   Warehouse Helper               22562         29382               38055   
122   Warehouse Helper               22033         28693               37163   
123   Warehouse Helper               22251         28976               37530   
124   Warehouse Helper               21814         28407               36793   
125   Warehouse Helper               21406         27877               36106   

    salary_type currency salary_duratio

In [29]:
salary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   job_designation         126 non-null    object             
 1   percentile25_salary     126 non-null    object             
 2   median_salary           126 non-null    object             
 3   percentile75_salary     126 non-null    object             
 4   salary_type             126 non-null    object             
 5   currency                126 non-null    object             
 6   salary_duration         126 non-null    object             
 7   location                126 non-null    object             
 8   last_reviewed_datetime  126 non-null    object             
 9   weblink                 126 non-null    object             
 10  job_desc                126 non-null    object             
 11  county                  126 non-null    objec

In [33]:
#Data Cleaning

#Splitting the 'location' feature into two new features 'county' and 'country'
location=salary_df['location'].str.split(",",expand=True) #expand=True will give the output in a dataframe instead of the default list
salary_df['county']=location[0]
salary_df['country']=location[1]

#Extracting only the date part from 'last_reviewed_datetime' and saving it to a new feature 'last_reviewed_date'
import datetime
salary_df['last_reviewed_date']=pd.to_datetime(salary_df['last_reviewed_datetime'].str.slice(0, 10))

#Final salary dataset after cleaning
salary_df

,job_designation,percentile25_salary,median_salary,percentile75_salary,salary_type,currency,salary_duration,location,last_reviewed_datetime,weblink,job_desc,county,country,last_reviewed_date
0,Data Architect II,42167,50115,58800,base,EUR,P1Y,"Dublin, Ireland",2021-03-19T00:00:00Z,https://www.salary.com/research/ie-salary/benc...,Contribute to an architecture roadmap covering...,Dublin,Ireland,2021-03-19
1,Data Architect II,39972,47506,55739,base,EUR,P1Y,"Cork, Ireland",2021-03-19T00:00:00Z,https://www.salary.com/research/ie-salary/benc...,Contribute to an architecture roadmap covering...,Cork,Ireland,2021-03-19
2,Data Architect II,39035,46393,54433,base,EUR,P1Y,"Galway, Ireland",2021-03-19T00:00:00Z,https://www.salary.com/research/ie-salary/benc...,Contribute to an architecture roadmap covering...,Galway,Ireland,2021-03-19
3,Data Architect II,39420,46850,54969,base,EUR,P1Y,"Limerick, Ireland",2021-03-19T00:00:00Z,https://www.salary.com/research/ie-salary/benc...,Contribute to an architecture roadmap covering...,Limerick,Ireland,2021-03-19
4,Data Architect II,38646,45930,53890,base,EUR,P1Y,"Shannon-West, Ireland",2021-03-19T00:00:00Z,https://www.salary.com/research/ie-salary/benc...,Contribute to an architecture roadmap covering...,Shannon-West,Ireland,2021-03-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,Warehouse Helper,22562,29382,38055,base,EUR,P1Y,"Cork, Ireland",2021-03-19T00:00:00Z,https://www.salary.com/research/ie-salary/alte...,When assigned to this particular warehouse dut...,Cork,Ireland,2021-03-19
122,Warehouse Helper,22033,28693,37163,base,EUR,P1Y,"Galway, Ireland",2021-03-19T00:00:00Z,https://www.salary.com/research/ie-salary/alte...,When assigned to this particular warehouse dut...,Galway,Ireland,2021-03-19
123,Warehouse Helper,22251,28976,37530,base,EUR,P1Y,"Limerick, Ireland",2021-03-19T00:00:00Z,https://www.salary.com/research/ie-salary/alte...,When assigned to this particular warehouse dut...,Limerick,Ireland,2021-03-19
124,Warehouse Helper,21814,28407,36793,base,EUR,P1Y,"Shannon-West, Ireland",2021-03-19T00:00:00Z,https://www.salary.com/research/ie-salary/alte...,When assigned to this particular warehouse dut...,Shannon-West,Ireland,2021-03-19
